In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
from dash import Dash, html, Output, Input
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.enrich import html, DashProxy
from dash_extensions.javascript import arrow_function, assign
# standalone map
import folium
from folium import plugins
# data manipulation
import numpy as np
import pandas as pd
import json
import geojson
import geopandas as gpd

data_directory = '/home/jan/Uni/DS-Project/data/Maps/'

In [3]:
# load germany maps & polygons
#bund = gpd.read_file(data_directory + 'Bundeskarte.geojson').to_crs(4326)
#land = gpd.read_file(data_directory + 'Laenderkarte.geojson').to_crs(4326)
kreis = gpd.read_file(data_directory + 'Kreiskarte.geojson').to_crs(4326)
polygon = gpd.read_file(data_directory + 'brandenburg_polygons.geojson').set_crs(25833, allow_override = True).to_crs(4326)

In [4]:
polygon_offset = polygon.copy().to_crs(3043)
polygon_offset['geometry'] = polygon_offset.geometry.buffer(-12, single_sided = True)
polygon_offset['area'] = polygon_offset['geometry'].area / 10**3
polygon_offset['geometry'] = polygon_offset.geometry.to_crs(4326)

In [5]:
# add random score between 0 and 10 to kreis
np.random.seed(123)
kreis.insert(
    13,
    "Score",
    np.random.uniform(0, 11, size=len(kreis))
)

In [6]:
'''
with open("kreis.json", "w") as f:
    f.write(kreis.to_json())
'''

In [7]:
kreis_path = "assets/kreis.json"
# load kreis
with open(kreis_path, 'r') as f:
    kreis = [f['properties'] for f in json.loads(f.read())['features']]

In [8]:
# define info box
def get_info(feature=None):
    header = [html.H4("Fitness für Ohren")]
    if not feature:
        return header + [html.P("Über Kreis schweben")]
    return header + [html.B(feature["properties"]["NAME_3"]), html.Br(),
                     "{:.2f} Lustnauer Score".format(feature["properties"]["Score"])]

classes = [0, 2, 4, 6, 8, 10]
colorscale = ['#ffd34c', '#e5f3e5', '#99d099', '#66b967', '#32a234', '#008b02']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")
# Create geojson.
geojson = dl.GeoJSON(url=f"/{kreis_path}",  # url to geojson file
                     options=dict(style=style_handle),  # how to style each polygon
                     zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")
# Create info control.
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1000"})
# app backend
app = Dash(prevent_initial_callbacks=True)
app.layout = html.Div([dl.Map(children=[dl.TileLayer(), geojson, colorbar, info])],
                      style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"}, id="map")


@app.callback(Output("info", "children"), [Input("geojson", "hover_feature")])
def info_hover(feature):
    return get_info(feature)

In [9]:
# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [04/Jan/2023 11:55:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:05] "GET /assets/dashExtensions_default.js?m=1672829694.0023456 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:08] "GET /assets/kreis.json HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:55:58] "POST /_